# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察標籤編碼與獨編碼熱的影響

# [作業重點]
- 回答在範例中的觀察結果
- 觀察標籤編碼與獨熱編碼, 在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 觀察範例，在房價預測中調整標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 方式，  
對於線性迴歸以及梯度提升樹兩種模型，何者影響比較大?

In [30]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

# 對 train、test 資料做合併與取出處理
train_Y = df_train['Survived']                                  # 取出目標值
ids = df_test['PassengerId']                                    # 取出乘客 ID
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)  # 去除目標值和乘客ID 欄位
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])                              # 合併 train、test 欄位
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [31]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':                               # 篩選出 type 為類別的欄位
        object_features.append(feature)                 # 把欄位名稱加入 list 中
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]                                # 只留下類別型欄位
df = df.fillna('None')                                  # NAN(缺值)填補 'None'
train_num = train_Y.shape[0]                            # 獲得 train 資料數
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)

In [32]:
# 標籤編碼 + 羅吉斯迴歸
start = time.time()             # 獲取開始時間


le = LabelEncoder()             # 簡寫 LabelEncoder()
df_temp = pd.DataFrame()        # 把 df_temp 變數設定為一DataFrame物件
for col in df.columns:
     df_temp[col] = le.fit_transform(df[col])                        # 進行 LabelEncoding 轉換
print(df_temp.head())   


train_X = df_temp[:train_num]                                        # 取出 train 資料
estimator = LogisticRegression()                                     # 使用線性回歸(羅吉斯回歸)
score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()    # 計算預測分數
print(f'\nscore : {score}')                                          # f'' : 為 f 字串
print(f'\ntrian_X_shape : {train_X.shape}')

print(f'\ntime_spend : {time.time()-start}')                         # 計算總共花費時間

   Name  Sex  Ticket  Cabin  Embarked
0   155    1     720    185         3
1   286    0     816    106         0
2   523    0     914    185         3
3   422    0      65     70         3
4    22    1     649    185         3

score : 0.7800138095537004

trian_X_shape : (891, 5)

time_spend : 0.12765765190124512


In [33]:
# 獨熱編碼 + 羅吉斯迴歸
start = time.time()                                                # 獲取開始時間


df_temp = pd.get_dummies(df)                                       # 進行 One-Hot-Encoding 轉換
train_X = df_temp[:train_num]                                      # 取出 train 資料


estimator = LogisticRegression()                                   # 使用羅吉斯回歸
score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()  # 預測分數計算
print(f'score : {score}')
print(f'\ntrian_X_shape : {train_X.shape}')                        # 可以發現 One-Hot-Encoding 的欄位數(特徵數)大增     

print(f'\ntime_spend : {time.time()-start}')                       # 計算總共花費時間

score : 0.8013307388111229

trian_X_shape : (891, 2429)

time_spend : 0.7819085121154785


## 觀察結果 : 

[ LabelEncoding / One-Hot-Encoding ]

特徵數(欄位數)  :   [ 同原欄位數   /  依照資料數大幅增加 ]

花費時間      :       [ 少        /        多 ]

預測分數      :      [ 較低       /       較高 ]